In [ ]:
from google.colab import files
data=files.upload()


Saving ETTh1.csv to ETTh1.csv
Saving ETTh2.csv to ETTh2.csv
Saving ETTm1.csv to ETTm1.csv
Saving ETTm2.csv to ETTm2.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

ettH1_data = pd.read_csv('ETTh1.csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')

etth1_data_original = ettH1_data[['date', 'HUFL']].copy()
etth1_data_original['date'] = pd.to_datetime(etth1_data_original['date'])

etth2_data_original = ettH2_data[['date', 'HULL']].copy()
etth2_data_original['date'] = pd.to_datetime(etth2_data_original['date'])

ettm1_data_original = ettM1_data[['date', 'HUFL']].copy()
ettm1_data_original['date'] = pd.to_datetime(ettm1_data_original['date'])

ettm2_data_original = ettM2_data[['date', 'HULL']].copy()
ettm2_data_original['date'] = pd.to_datetime(ettm2_data_original['date'])

ettH1_data['date'] = etth1_data_original['date']
ettH2_data['date'] = etth2_data_original['date']
ettM1_data['date'] = ettm1_data_original['date']
ettM2_data['date'] = ettm2_data_original['date']

etth1_data = ettH1_data[['date', 'HUFL']]
etth1_data['date'] = pd.to_datetime(etth1_data['date'])
etth1_data.set_index('date', inplace=True)
etth1_values = etth1_data.values.astype(float)

etth2_data = ettH2_data[['date', 'HULL']]
etth2_data['date'] = pd.to_datetime(etth2_data['date'])
etth2_data.set_index('date', inplace=True)
etth2_values = etth2_data.values.astype(float)

ettm1_data = ettM1_data[['date', 'HUFL']]
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])
ettm1_data.set_index('date', inplace=True)
ettm1_values = ettm1_data.values.astype(float)

ettm2_data = ettM2_data[['date', 'HULL']]
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])
ettm2_data.set_index('date', inplace=True)
ettm2_values = ettm2_data.values.astype(float)

sequence_length = 50

scaler_etth1 = MinMaxScaler()
scaler_etth2 = MinMaxScaler()
scaler_ettm1 = MinMaxScaler()
scaler_ettm2 = MinMaxScaler()

etth1_scaled = scaler_etth1.fit_transform(etth1_values)
etth2_scaled = scaler_etth2.fit_transform(etth2_values)
ettm1_scaled = scaler_ettm1.fit_transform(ettm1_values)
ettm2_scaled = scaler_ettm2.fit_transform(ettm2_values)

def create_lstm_model(sequence_length, num_features):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, num_features)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_multivariate_lstm(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :])
        y.append(data[i + sequence_length, 0])

    X, y = np.array(X), np.array(y)
    model = create_lstm_model(sequence_length, data.shape[1])
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, data.shape[1]))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)

predictions_etth1 = train_multivariate_lstm(etth1_scaled, sequence_length)
predictions_etth2 = train_multivariate_lstm(etth2_scaled, sequence_length)
predictions_ettm1 = train_multivariate_lstm(ettm1_scaled, sequence_length)
predictions_ettm2 = train_multivariate_lstm(ettm2_scaled, sequence_length)

combined_predictions = np.column_stack((predictions_etth1, predictions_etth2, predictions_ettm1, predictions_ettm2))
print(combined_predictions)


<ipython-input-5-0d2725c91d8d>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth1_data['date'] = pd.to_datetime(etth1_data['date'])
<ipython-input-5-0d2725c91d8d>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth2_data['date'] = pd.to_datetime(etth2_data['date'])
<ipython-input-5-0d2725c91d8d>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

Epoch 1/5
543/543 [==============================] - 32s 51ms/step - loss: 0.0140
Epoch 2/5
543/543 [==============================] - 28s 51ms/step - loss: 0.0041
Epoch 3/5
543/543 [==============================] - 29s 54ms/step - loss: 0.0030
Epoch 4/5
543/543 [==============================] - 28s 51ms/step - loss: 0.0029
Epoch 5/5
1/1 [==============================] - 0s 25ms/step
Epoch 1/5
543/543 [==============================] - 32s 53ms/step - loss: 0.0031
Epoch 2/5
543/543 [==============================] - 29s 54ms/step - loss: 0.0014
Epoch 3/5
543/543 [==============================] - 30s 55ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 30s 56ms/step - loss: 0.0010
Epoch 5/5
1/1 [==============================] - 0s 29ms/step
Epoch 1/5
2176/2176 [==============================] - 130s 58ms/step - loss: 0.0030
Epoch 2/5
2176/2176 [==============================] - 127s 58ms/step - loss: 0.0011
Epoch 3/5
2176/2176 [=============================